In [14]:
import pandas as pd

In [15]:
# Se importan sólo las columnas necesarias.

user_reviews = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\user_reviews.parquet",columns=['item_id','sentiment_analysis'])
user_reviews.head()

,item_id,sentiment_analysis
0,1250,2
1,22200,1
2,43110,2
3,251610,1
4,227300,1


In [16]:
# Se importan sólo las columnas necesarias.

steam_games = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\steam_games.parquet",columns=['developer','id'])

steam_games.head()

,developer,id
0,Kotoshiro,761140.0
1,Secret Level SRL,643980.0
2,Poolians.com,670290.0
3,彼岸领域,767400.0
4,Trickjump Games Ltd,772540.0


In [18]:
# Se unen los datasets previamente recopilados en un dataframe.

data = pd.merge(user_reviews,steam_games, left_on='item_id',right_on='id',how='inner')

data.head()

,item_id,sentiment_analysis,developer,id
0,1250,2,Tripwire Interactive,1250
1,22200,1,ACE Team,22200
2,227300,1,SCS Software,227300
3,239030,2,3909,239030
4,370360,2,Zachtronics,370360


In [19]:
data = data[['developer','sentiment_analysis']]  # Se flitran sólo las variables necesarias.

data.head()

,developer,sentiment_analysis
0,Tripwire Interactive,2
1,ACE Team,1
2,SCS Software,1
3,3909,2
4,Zachtronics,2


In [20]:
count_neg = data[data['sentiment_analysis']==0].groupby('developer')['sentiment_analysis'].count().reset_index() # Se cuentan los elementos negativos por desarrollador

count_pos = data[data['sentiment_analysis']==2].groupby('developer')['sentiment_analysis'].count().reset_index() # Se cuentan los elementos positivos por desarrollador

In [21]:
# Los dos conteos previamente obtenidos se unen al dataframe "data".

developer_reviews_analysis = pd.merge(data,count_pos,on='developer').rename(columns={'sentiment_analysis_x':'sentiment_analysis','sentiment_analysis_y':'count_pos'})

developer_reviews_analysis = pd.merge(developer_reviews_analysis,count_neg, on='developer').rename(columns={'sentiment_analysis_x':'sentiment_analysis','sentiment_analysis_y':'count_neg'})

developer_reviews_analysis.head()

,developer,sentiment_analysis,count_pos,count_neg
0,Tripwire Interactive,2,174,84
1,ACE Team,1,2,3
2,SCS Software,1,38,14
3,3909,2,21,6
4,Zachtronics,2,3,1


In [22]:
# Se filtran sólo las variables necesarias y se define a "developer" como el nuevo índice.

developer_reviews_analysis = developer_reviews_analysis[['developer','count_pos','count_neg']].set_index('developer')

developer_reviews_analysis.head()

,count_pos,count_neg
developer,,
Tripwire Interactive,174,84
ACE Team,2,3
SCS Software,38,14
3909,21,6
Zachtronics,3,1


In [23]:
developer_reviews_analysis.drop_duplicates(inplace=True)

In [24]:
developer_reviews_analysis.to_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\Apis\developer_reviews_analysis\dra.parquet",compression='snappy')

In [25]:
developer_reviews_analysis = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\Apis\developer_reviews_analysis\dra.parquet")

developer_reviews_analysis.head()

,count_pos,count_neg
developer,,
Tripwire Interactive,174,84
ACE Team,2,3
SCS Software,38,14
3909,21,6
Zachtronics,3,1
